In [88]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.11.0
Running on TPU  ['10.4.48.250:8470']


In [89]:
# importing necessary packages
from transformers import Trainer, TrainingArguments  # importing traine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import re
import torch

In [90]:
!pip install transformers # installing transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [91]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)
# creating a DistilBertForSequenceClassification pretarained model

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/mo

In [92]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# creating a tokenizer

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_tok

In [ ]:
data

In [93]:
data = pd.read_csv('/content/Corona_NLP_test.csv',encoding='latin-1') # reading a csv file 
train = data.iloc[:,[4,5]] # creating required dataframe to perform multioutput-text classification

In [94]:
# import natural language tool kit
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [95]:
encode = LabelEncoder()
train['Sentiment'] = encode.fit_transform(train['Sentiment'])

<ipython-input-95-23eb82dcdd68>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Sentiment'] = encode.fit_transform(train['Sentiment'])


Preprocess the text data by cleaning it, removing stopwords, punctuations and other irrelevant characters.


In [96]:

corpus = []
for i in range(len(train)):
  tra = train['OriginalTweet'][i].lower()
  tra = re.sub('[^a-z-A-Z]',' ',tra)
  tra = tra.split()
  tra = [tr for tr in tra if tr not in set(stopwords.words('english'))]
  tra = ' '.join(tra)
  corpus.append(tra)

y = list(train['Sentiment'])

In [97]:
# splitting the the data into train and test
x_train,x_test,y_train,y_test = train_test_split(corpus,train['Sentiment'],random_state=0)

# creating tokens using tokenizer
X_train_tokenized = tokenizer(x_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(x_test, padding=True, truncation=True, max_length=512)

X_train_tokenized.keys()

print(X_train_tokenized['attention_mask'][0])


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [98]:
!pip install torch
# installing torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [99]:
import torch

In [100]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, list(y_train))

val_dataset = Dataset(X_val_tokenized, list(y_test))


Use the Hugging Face library to fine-tune a pre-trained model such as BERT on the classification task.


In [101]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# making compute metrics for model evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [102]:
# Defining training arguments
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=16,   # batch size for evaluation


)
# defining the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [103]:
# training the trainer
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2848
  Num Epochs = 1
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 285
  Number of trainable parameters = 66957317


Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
# evaluate the model
trainer.evaluate()

In [104]:
# importing test data
test_data = pd.read_csv('/content/Corona_NLP_train.csv',encoding='latin-1') 


In [105]:
# trkin only 1100 sample for prediction
test_data = test_data.iloc[0:1100]

In [106]:
# taking  OriginalTweet	Sentiment from test data for prediction
test_val = test_data.iloc[:,[4,5]]

In [107]:
test_data.head(1)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral


In [108]:
# cleaning tje data and removing stopwords and special charecters
pre = []
for i in range(len(test_val)):
  test = test_val['OriginalTweet'][i].lower()
  test = re.sub('[^a-z-A-Z]',' ',test)
  test = test.split()
  test = [tes for tes in test if tes not in set(stopwords.words('english'))]
  test = ' '.join(test)
  pre.append(test)

In [109]:
# encoding the sentiment column in test data
test_val['Sentiment'] = encode.fit_transform(test_val['Sentiment'])

<ipython-input-109-affbddb9c2b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_val['Sentiment'] = encode.fit_transform(test_val['Sentiment'])


In [112]:
y = test_val['Sentiment']

In [111]:
# convert the pre data into tokens using tokenizer
test = tokenizer(pre, padding=True, truncation=True, max_length=512)


In [113]:
# creating torch data
test_val_dataset = Dataset(test,list(y))

In [114]:
test_val_dataset

In [115]:
# predicting the output using trainer
output=trainer.predict(test_val_dataset)[1]

***** Running Prediction *****
  Num examples = 1100
  Batch size = 16


Step,Training Loss


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [116]:
from sklearn.metrics import accuracy_score,classification_report

In [117]:
#checking for the accuracy of the model
accuracy_score(output,y)

1.0

# predicting with sample data

In [119]:
data['Sentiment'].value_counts()

Negative              1041
Positive               947
Neutral                619
Extremely Positive     599
Extremely Negative     592
Name: Sentiment, dtype: int64

In [ ]:
Conclusion:
In conclusion, a pre-trained DistilBERT model is fine-tuned on the COVID-19 tweet dataset to classify the tweets into one of the five different sentiments. The model is evaluated using accuracy, precision, recall, and F1-score, which helps in assessing the performance of the model on the dataset. The preprocessing steps taken help in reducing the noise in the dataset, and the DistilBERT model architecture provides the probability distribution over the different sentiments.

In [120]:
examples = [
    "This is a positive example.",
    "This is a negative example.",
    "I'm not sure how to classify this one.",
    "He is very dangerous fellow"
]

# Tokenize the examples and convert them to PyTorch tensors
inputs = tokenizer(examples, padding=True, truncation=True, return_tensors="pt")

# Make predictions with the model
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=1)

# Print the predicted labels
labels = ["negative", "positive","Neutral","Extremely Positive","Extremely Negative"]
for example, prediction in zip(examples, predictions):
    print(f"Example: {example} | Predicted label: {labels[prediction]}")

Example: This is a positive example. | Predicted label: Neutral
Example: This is a negative example. | Predicted label: Neutral
Example: I'm not sure how to classify this one. | Predicted label: Neutral
Example: He is very dangerous fellow | Predicted label: Neutral
